# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import warnings
warnings.filterwarnings("ignore")

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,11.01,76,96,0.45,NZ,1701792563
1,1,adamstown,-25.0660,-130.1015,22.40,82,100,6.19,PN,1701792563
2,2,thompson,55.7435,-97.8558,-12.29,94,66,2.36,CA,1701792563
3,3,hithadhoo,-0.6000,73.0833,28.23,73,41,2.87,MV,1701792563
4,4,edinburgh of the seven seas,-37.0676,-12.3116,15.34,95,100,5.09,SH,1701792564


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
#%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 600,
    frame_height = 500,
    size = 'Humidity',
    scale = .8,
    color = 'City'
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_vacation = city_data_df.loc[(city_data_df['Max Temp']<= 27)\
                                  & (city_data_df['Max Temp']>= 21)\
                                    & (city_data_df['Wind Speed']<= 4.5)\
                                    & (city_data_df['Cloudiness'] == 0), :]

# Drop any rows with null values
ideal_vacation.dropna()

# Display sample data
ideal_vacation.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
38,38,salalah,17.0151,54.0924,26.05,73,0,1.54,OM,1701792414
53,53,kailua-kona,19.6406,-155.9956,22.50,76,0,1.54,US,1701792286
121,121,as sulayyil,20.4607,45.5779,21.14,20,0,4.16,SA,1701792586
279,279,carlsbad,33.1581,-117.3506,21.61,49,0,0.00,US,1701792409
327,327,alice springs,-23.7000,133.8833,21.75,49,0,0.00,AU,1701792638


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_vacation.iloc[:,[1,8,2,3,5]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
38,salalah,OM,17.0151,54.0924,73,
53,kailua-kona,US,19.6406,-155.9956,76,
121,as sulayyil,SA,20.4607,45.5779,20,
279,carlsbad,US,33.1581,-117.3506,49,
327,alice springs,AU,-23.7000,133.8833,49,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {'categories' : 'accommodation.hotel',\
          'filter' : '', \
          'bias' : '', \
          'apiKey' : geoapify_key, \
          'limit' : 1   
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row['Lng']
    lat = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
salalah - nearest hotel: Muscat International Hotel
kailua-kona - nearest hotel: Kona Seaside Hotel
as sulayyil - nearest hotel: No hotel found
carlsbad - nearest hotel: Carlsbad Inn Beach Resort
alice springs - nearest hotel: Aurora Alice Springs
sakakah - nearest hotel: فندق النزل
tamanrasset - nearest hotel: فندق الأمان
cabo san lucas - nearest hotel: Comfort Rooms
macau - nearest hotel: 富華粵海酒店 Fu Hua Guang Dong Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
38,salalah,OM,17.0151,54.0924,73,Muscat International Hotel
53,kailua-kona,US,19.6406,-155.9956,76,Kona Seaside Hotel
121,as sulayyil,SA,20.4607,45.5779,20,No hotel found
279,carlsbad,US,33.1581,-117.3506,49,Carlsbad Inn Beach Resort
327,alice springs,AU,-23.7000,133.8833,49,Aurora Alice Springs
357,sakakah,SA,29.9697,40.2064,32,فندق النزل
375,tamanrasset,DZ,22.7850,5.5228,9,فندق الأمان
414,cabo san lucas,MX,22.8909,-109.9124,50,Comfort Rooms
532,macau,MO,22.2006,113.5461,78,富華粵海酒店 Fu Hua Guang Dong Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
#%%capture --no-display

# Configure the map plot
plot_2 = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles='OSM',
    frame_width = 600,
    frame_height = 500,
    color= 'City',
    alpha = 0.5,
    scale = 0.8,
    hover_cols=['Hotel Name', 'Country']
)

# Display the map
plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)